In [3]:
# pip install PyGithub
#pip install python-dotenv

In [6]:
import re
import pandas as pd
import re
import pandas as pd
from github import Github
import os
import numpy as np

from dotenv import load_dotenv
import os

## Section 3.1.1

https://github.com/Mirantis
https://hg.mozilla.org/
https://git.openstack.org/cgit
https://gerrit.wikimedia.org/r/#/admin/projects/

In [7]:

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Récupérer la clé API
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")


g = Github(GITHUB_TOKEN)

# Objectifs de récupérations de repos par organisation
REPOS_PER_ORGANIZATION = {
    'Mozilla': 1594,
    'Mirantis': 26,
    'Openstack': 1253,
    'Wikimedia': 1638
}


#Critère sur l'activité
MIN_COMMITS_PER_MONTH = 2
# Critère sur le pourcentage de fichiers IAC
MIN_IAC_SCRIPT_PERCENTAGE = 11  

# Extensions typiques de fichiers IaC
IAC_EXTENSIONS = ['.yaml', '.yml', '.json', '.sh', '.tf', '.pp']  


In [2]:

# Fonction pour vérifier si un dépôt est éligible
def is_eligible_repo(repo):
    try:
        # Calculer le nombre de commits par mois
        commits = repo.get_commits().totalCount
        months = (repo.updated_at - repo.created_at).days / 30
        if commits / months < MIN_COMMITS_PER_MONTH:
            return False

        # Analyser la proportion des fichiers IaC
        contents = repo.get_contents("")
        total_files = len(contents)
        iac_files = sum(1 for file in contents if any(file.path.endswith(ext) for ext in IAC_EXTENSIONS))

        if (iac_files / total_files) * 100 < MIN_IAC_SCRIPT_PERCENTAGE:
            return False

        return True
    except:
        return False

# Fonction principale pour collecter les dépôts
def collect_repos(organization_name, repo_limit):
    organization = g.get_organization(organization_name)
    repos = organization.get_repos()
    collected_repos = []
    nb_collected = 0
    for repo in repos :
        if nb_collected >= repo_limit:
            break
        if is_eligible_repo(repo):
            nb_collected +=1
            collected_repos.append(repo.clone_url)
    
    return collected_repos




In [3]:
# Collecte des dépôts pour chaque organisation
for org, repo_limit in REPOS_PER_ORGANIZATION.items():
    eligible_repos = collect_repos(org, repo_limit)
    print(f"{org}: {len(eligible_repos)} dépôts éligibles")
    # Sauvegarder les dépôts dans un fichier ou une base de données
    with open(f'{org}_eligible_repos.txt', 'w') as f:
        for repo_url in eligible_repos:
            f.write(f"{repo_url}\n")

Wikimedia: 740 dépôts éligibles


## Section 3.1.2

In [4]:

# Fonction pour vérifier si un fichier est un script IaC
def is_iac_file(file_path):
    return any(file_path.endswith(ext) for ext in IAC_EXTENSIONS)

# Nouvelle fonction pour extraire les identifiants d'issues
def extract_issue_ids(commit_message):
    # ex : "Fix the color (#400)"
    pattern = r'(?i)(?:fix(?:e[sd])?|close[sd]?|resolve[sd]?|merge pull request|pull request|pr|issue|bug)[\s:#-]*#?(\d+)|\(#(\d+)\)|#(\d+)'
    matches = re.findall(pattern, commit_message)
    issue_numbers = []
    for match in matches:
        # Chaque match est un tuple contenant les groupes capturés
        for group in match:
            if group and group.isdigit():
                issue_numbers.append(group)
    return issue_numbers

# Fonction pour extraire les commits modifiant des scripts IaC
def extract_iac_commits(repo):
    iac_commits = []
    commits = repo.get_commits()
    count = 0  # Compteur pour limiter le nombre de commits traités

    for commit in commits:
        if count >= 20:  
            break
        try:
            # Vérifie si au moins un fichier modifié est un script IaC
            files = commit.files  # Appel API
            if any(is_iac_file(file.filename) for file in files):
                iac_commits.append(commit)
            count += 1
        except Exception as e:
            print(f"Erreur lors de l'accès aux fichiers du commit {commit.sha}: {e}")
            continue

    return iac_commits

# Fonction pour extraire les messages de commit et résumer les incidents
def process_commit_messages(iac_commits, repo):
    commit_data = []

    for commit in iac_commits:
        commit_message = commit.commit.message
        
        # Extraire les identifiants d'issues du message de commit
        issue_ids = extract_issue_ids(commit_message)
        
        issue_summaries = []

        if issue_ids and repo.has_issues:
            for issue_id in issue_ids:
                issue_title = None
                # Tenter de récupérer l'issue
                try:
                    issue = repo.get_issue(int(issue_id))
                    issue_title = issue.title
                except Exception as e:
                   
                    # Tenter de récupérer la pull request
                    try:
                        pull = repo.get_pull(int(issue_id))
                        issue_title = pull.title
                    except Exception as e:
                       
                        issue_title = f"Aucun résumé trouvé pour l'issue ou la pull request {issue_id}"
                issue_summaries.append(issue_title)
        else:
            issue_summaries.append("Aucune issue référencée ou issues non activées")

        # Construction du message étendu (XCM)
        xcm = {
            'repo_name': repo.full_name,
            'commit_message': commit_message,
            'issue_summaries': "; ".join(issue_summaries),
            'commit_url': commit.html_url
        }

        commit_data.append(xcm)

    return commit_data


# Fonction principale pour traiter les commits d'un dépôt et construire un DataFrame
def process_repo_commits(repo_url):
    repo_name = repo_url.split('/')[-1]  # Extraire le nom du repo depuis l'URL
    
    repo_full_name = repo_url.replace("https://github.com/", "").replace(".git", "")
    repo = g.get_repo(repo_full_name)
    iac_commits = extract_iac_commits(repo)
    commit_data = process_commit_messages(iac_commits, repo)

    return commit_data

# Lire les fichiers contenant les URLs des dépôts et collecter les informations
def process_repos_from_file(file_path):
    collected_data = []

    # Lire le fichier contenant les URLs des dépôts
    with open(file_path, 'r') as file:
        repo_urls = file.readlines()
    i = 0
    # Pour chaque URL de dépôt, traiter les commits et collecter les données
    for repo_url in repo_urls:
        print(f"Processing {i}/{len(repo_urls)}")
        i +=1
        
        repo_url = repo_url.strip()  # Supprimer les espaces inutiles
        try:
            repo_commit_data = process_repo_commits(repo_url)
            collected_data.extend(repo_commit_data)
        except Exception as e:
            print(f"Erreur pour le dépôt {repo_url}: {e}")

    # Convertir les données collectées en DataFrame Pandas
    df = pd.DataFrame(collected_data)
    return df



In [9]:

file_paths = ['Mirantis_eligible_repos.txt', 'Mozilla_eligible_repos.txt', 'Openstack_eligible_repos.txt', 'Wikimedia_eligible_repos.txt']


In [ ]:


all_data = []
for file_path in file_paths:
    print(f"Processing {file_path}")
    repo_data = process_repos_from_file(file_path)
    all_data.append(repo_data)
    

# Combiner toutes les données dans un seul DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Afficher ou sauvegarder le DataFrame
final_df.to_csv('iac_commit_data.csv', index=False)


### Stats


In [3]:
df = pd.read_csv("iac_commit_data.csv",sep=',',header=0)

In [6]:
df.head(400)

,repo_name,commit_message,issue_summaries,commit_url
0,Mirantis/puppetlabs-openstack,Merge pull request #137 from aimonb/master\n\n...,Aucune issue référencée ou issues non activées,https://github.com/Mirantis/puppetlabs-opensta...
1,Mirantis/puppetlabs-openstack,add nova-volume class.,Aucune issue référencée ou issues non activées,https://github.com/Mirantis/puppetlabs-opensta...
2,Mirantis/puppetlabs-openstack,Merge pull request #135 from aimonb/master\n\n...,Aucune issue référencée ou issues non activées,https://github.com/Mirantis/puppetlabs-opensta...
3,Mirantis/puppetlabs-openstack,FIX: add missing arg. ADD: tests,Aucune issue référencée ou issues non activées,https://github.com/Mirantis/puppetlabs-opensta...
4,Mirantis/puppetlabs-openstack,Merge pull request #130 from bodepd/aimon_spec...,Aucune issue référencée ou issues non activées,https://github.com/Mirantis/puppetlabs-opensta...
...,...,...,...,...
395,mozilla/login.webmaker.org,Merge pull request #387 from ryanwarsaw/replac...,"Replace bcrypt library with bcrypt.js, Closes ...",https://github.com/mozilla/login.webmaker.org/...
396,mozilla/login.webmaker.org,Merge pull request #388 from ryanwarsaw/remove...,Redundant dependency: newrelic,https://github.com/mozilla/login.webmaker.org/...
397,mozilla/MakeAPI,1.2.28,Aucune issue référencée ou issues non activées,https://github.com/mozilla/MakeAPI/commit/dcd3...
398,mozilla/MakeAPI,Add timeout,Aucune issue référencée ou issues non activées,https://github.com/mozilla/MakeAPI/commit/db7c...


In [23]:
dict = np.zeros(shape=[4])
for file_name,i in zip(file_paths,range(0,4)):
    with open(file_name, 'r') as file:
        lignes = file.readlines()
        nb = len(lignes)
        dict[i] = nb
    
        
tot = sum(dict)
print(f"On a récupérer {tot} repos éligibles.")

On a récupérer 1220.0 repos éligibles.


In [8]:
print(f"Nombre de commits : {str(len(df))}")

Nombre de commits : 14310
